In [ ]:
import os
import shutil
import nibabel as nib
import numpy as np
from glob import glob
from tqdm import tqdm_notebook

In [ ]:
#CT1 resampling

In [ ]:
srcpath = '/home/sc-mlsg/Downloads/preprocessing_data_platform/perfusion/Real_Extract/set_25/original_data'
dest_dir = '/home/sc-mlsg/Downloads/preprocessing_data_platform/perfusion/Real_Extract/set_25/preprocessing_data/CT1_resample'
os.makedirs(dest_dir, exist_ok = True)
os.makedirs(srcpath, exist_ok = True)

In [ ]:
srcpath = '/home/sc-mlsg/Downloads/preprocessing_data_platform/perfusion/Real_Extract/set_40/original_data/CT1'
for (path, dirs, files) in os.walk(srcpath):
    print(path, dirs, files)

In [ ]:
total = 0

for (path, dirs, files) in os.walk(srcpath):
    for file in sorted(files):
        total += 1
        print(total, file)
        in_path = os.path.join(path, file)
        
        
        file_name = file.split('.')[0]
        new_filename = "{}_resample.nii.gz".format(file_name)
        print(new_filename)
        print()
        
        outPath = os.path.join(dest_dir, new_filename)
        
        if os.path.exists(outPath):
            print("File already exists!!")
            continue
            
        !ResampleImage 3 {in_path} {outPath} 1x1x1
            

# mask_resampling

In [ ]:
dest_dir = '/home/sc-mlsg/Downloads/preprocessing_data_platform/perfusion/Real_Extract/set_25/preprocessing_data/mask_resample'
os.makedirs(dest_dir, exist_ok = True)

In [ ]:
srcpath = '/home/sc-mlsg/Downloads/preprocessing_data_platform/perfusion/Real_Extract/set_25/original_data/mask'
for (path, dirs, files) in os.walk(srcpath):
    print(path, dirs, files)

====================== check label

In [ ]:
total = 0

for (path, dirs, files) in os.walk(srcpath):
    if files:
        total += 1
        pt = os.path.basename(path)
        print("# {} : resample mask {}".format(total, pt))
        pt_dir = os.path.join(dest_dir, pt)
        os.makedirs(pt_dir, exist_ok = True)
        
        
        for file in sorted(files):
            print(file)
            in_path = os.path.join(path,file)

            img = nib.load(in_path)
            img_data = img.get_fdata()
            
            print(np.unique(img_data, return_counts= True))
            print()
        
        print()

==================================

All label's components are zero and one

In [ ]:
total = 0

for (path, dirs, files) in os.walk(srcpath):
    if files:
        total += 1
        pt = os.path.basename(path)
        print("# {} : resample mask {}".format(total, pt))
        pt_dir = os.path.join(dest_dir, pt)
        os.makedirs(pt_dir, exist_ok = True)
        
        
        for file in sorted(files):
            print(file)
            in_path = os.path.join(path,file)

            new_filename = "{}_resample.nii.gz".format(file.split('.')[0])
            print(new_filename)

            outPath = os.path.join(pt_dir, new_filename)

            if os.path.exists(outPath):
                print("File already exists!!")
                continue

            !ResampleImage 3 {in_path} {outPath} 1x1x1
        
        print()

# make clean mask

In [ ]:
dest_dir = '/home/sc-mlsg/Downloads/preprocessing_data_platform/perfusion/Real_Extract/set_25/preprocessing_data/mask_resample_clean'
os.makedirs(dest_dir, exist_ok = True)

In [ ]:
srcpath =  '/home/sc-mlsg/Downloads/preprocessing_data_platform/perfusion/Real_Extract/set_25/preprocessing_data/mask_resample'
for (path, dirs, files) in os.walk(srcpath):
    print(path, dirs, files)

================= check resample mask

In [ ]:
total = 0

for (path, dirs, files) in os.walk(srcpath):
    if files:
        total += 1
        pt = os.path.basename(path)
        print("# {} : resample mask {}".format(total, pt))
        pt_dir = os.path.join(dest_dir, pt)
        os.makedirs(pt_dir, exist_ok = True)
        
        
        for file in files:
            print(file)
            in_path = os.path.join(path,file)
            
            mask_img = nib.load(in_path)
            mask_img_data = mask_img.get_fdata()
            
            print(np.unique(mask_img_data , return_counts = True))
            for i in np.unique(mask_img_data):
                print(i)
            
        print()

===========================

In [ ]:
total = 0

for (path, dirs, files) in os.walk(srcpath):
    if files:
        total += 1
        pt = os.path.basename(path)
        print("# {} : resample mask {}".format(total, pt))
        pt_dir = os.path.join(dest_dir, pt)
        os.makedirs(pt_dir, exist_ok = True)
        
        
        for file in files:
            print(file)
            in_path = os.path.join(path,file)
            
            mask_img = nib.load(in_path)
            mask_img_data = mask_img.get_fdata()
            
            new_mask_data = np.where(mask_img_data > 0.5 , 1, 0).astype('uint8')
            print(new_mask_data.dtype)
            print(np.unique(new_mask_data , return_counts= True))
            
            new_mask = nib.Nifti1Image(new_mask_data , mask_img.affine, mask_img.header)
            new_mask.header.set_data_dtype(np.uint8)

            new_filename = "{}_clean.nii.gz".format(file.split('.')[0])
            print(new_filename)
            print()

            outPath = os.path.join(pt_dir, new_filename)

            if os.path.exists(outPath):
                print("File already exists!!")
                continue
            
            nib.save(new_mask, outPath)
            
        print()

check if there is no labeling....label_2

# Shape check CT1 & mask

In [ ]:
CT1_dir = '/home/sc-mlsg/Downloads/preprocessing_data_platform/perfusion/Real_Extract/set_25/preprocessing_data/CT1_resample'
mask_dir = '/home/sc-mlsg/Downloads/preprocessing_data_platform/perfusion/Real_Extract/set_25/preprocessing_data/mask_resample_clean' 

In [ ]:
pt_lst = os.listdir(mask_dir)
pt_lst = sorted([int(pt) for pt in pt_lst])
pt_lst = [str(pt) for pt in pt_lst]
pt_lst

In [ ]:
total = 0

for pt in pt_lst:
    total += 1
    print("# {} : check in {}".format(total, pt))
    
    CT1_path = glob(os.path.join(CT1_dir, "*{}*".format(pt)))
    mask_1_path = glob(os.path.join(mask_dir, pt, '*label_1*'))
    mask_2_path = glob(os.path.join(mask_dir, pt, '*label_2*'))
    
    print(CT1_path)
    print(mask_1_path)
    print(mask_2_path)
    
    if len(CT1_path) != 1:
        print("There is no CT1....")
        
    if len(mask_1_path) != 1:
        print("There is no label 1....")
        
        
    if len(mask_2_path) != 1:
        print("There is no label 2....")
        
        
    total_lst = CT1_path + mask_1_path + mask_2_path
    print()
#     print(total_lst)
    
    for file in total_lst:
        img = nib.load(file)
        print("check in {} : {}".format(os.path.basename(file), img.shape))
        # print(img.header)
        
    print()
    
    
    